# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [105]:
#Your code here
import pandas as pd
import numpy as np
import scipy.stats as stats

In [2]:
df = pd.read_csv("homepage_actions.csv")

In [83]:
df.head()
len(df)

8188

In [47]:
view_click = set(df[df['action']=='view']['id']) & set(df[df['action']=='click']['id'])
print("Number of Viewers Who Also Clicked: {}".format(len(view_click)))

Number of Viewers Who Also Clicked: 1860


In [53]:
print("Number of Clickers: {}".format(len(set(df[df['action']=='click']['id']))))

Number of Clickers: 1860


In [42]:
print("Number of Duplicated IDs: {}".format(df.id.duplicated().sum()), "\n"
      "Number of IDs Duplicated More Than Once: {}".format((df[df['id'].duplicated()==True].id.value_counts()>1).sum()))

Number of Duplicated IDs: 1860 
Number of IDs Duplicated More Than Once: 0


In [91]:
df_exp = df[df['group']=='experiment']
len(df_exp)

3924

In [92]:
df_con = df[df['group']=='control']
len(df_con)

4264

In [109]:
print("Number of IDs in Both the Control and Experimental Groups: {}".format(len(set(df_exp['id']) & set(df_con['id']))))

Number of IDs in Both the Control and Experimental Groups: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [75]:
#Your code here
from statsmodels.stats.proportion import proportions_ztest

In [93]:
nobs = [len(df_con['id'].unique()), len(df_exp['id'].unique())]
counts = [len(df_con[df_con['action']=='click']), len(df_exp[df_exp['action']=='click'])]

proportions_ztest(counts, nobs, value=0, alternative='smaller')

(-2.618563885349469, 0.004415037788297902)

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [112]:
#Your code here
print(counts, nobs)
con_rate = 932/3332
expected_exp_rate = con_rate*2996
expected_exp_rate


[932, 928] [3332, 2996]


838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [106]:
#Your code here
n=2996
p=con_rate
std=np.sqrt(n*p*(1-p))
std
z = (928-expected_exp_rate)/std

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [108]:
#Your code here
print(1-stats.norm.cdf(z))

0.00012486528006949715


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: Yes, the result roughly matches that of the previous statistical test. That said, this is a very convoluted why to check the hypothesis test results and it is not at all clear that this method is an accurate way to check the results. 

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.